### 연속체인 + FewShotChatMessagePrompt 예제

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage

# Custom Output Parser
def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content


chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


examples = [
    {
        "intent": "재생",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        넷플릭스, 라디오, 미디어 통합검색, 실시간채널 검색/재생, 음악 검색/재생
        """,
    },
    {
        "intent": "경로",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        내비정보, 대중교통, 버스, 택시, 지하철
        """,
    },
    {
        "intent": "생활 정보",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        날씨 정보 요청, 뉴스 브리핑, 뉴스속보 요청, 환율 정보 요청, 프로야구
        """,
    },
    {
        "intent": "채팅",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        가이드 발화, 감성채팅, 미분류, 미응대, 서비스 제안
        """,
    },
    {
        "intent": "일정",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        일정목록 입력, 일정목록 조회
        """,
    },
    {
        "intent": "메모",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        리마인더, 음성 메모, 음성 메시지
        """,
    },
    {
        "intent": "법률",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        법제처, 기본법령, 국가법령정보센터, 자치법규정보시스템, 한국법령정보원, 헌법재판소, 정부24, 입법 동향
        """,
    },
]


to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
  vectorstore=vectorstore,
  k=2 # 생성할 예제 개수입니다.
)


intent_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 대한 의도를 정확하게 파악할 수 있는 전문가입니다. 질문이 오면 해당 질문의 맥락을 파악하여 그 의도를 대표할 수 있는 한 개의 단어로 대답해주세요."),
    ("human", "{question}")
])

intent_chain = intent_prompt | chat | parse


example_intent_prompt = ChatPromptTemplate.from_messages([
    ("human", "{intent}"),
    ("ai", "{answer}")
])


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_intent_prompt,
)


small_intent_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 입력 받은 단어를 해석하여 문자열 list를 생성하는 시스템입니다. 입력 받은 단어가 예시와 0.7 이상의 유사성을 띄면 해당하는 예시의 값을 comma로 구분하여 list로 만들어 반환해주세요. 만약 유사성이 0.7 미만의 낮은 단어로 판단되는 경우 자체적으로 관련된 단어를 추론하여 최대 5개의 단어를 담고 있는 list를 생성해주세요. 정확하게 list까지만 생성해서 답변해주고, 나머지 다른 답변은 필요하지 않습니다."),
    few_shot_prompt,
    ("human", "{intent}")
])


small_intent_chain = small_intent_prompt | chat


final_chain = {"intent": intent_chain} | small_intent_chain



# result = final_chain.invoke({
#     "question": "서울역에서 삼성역 가는 방법 알려줘."
# })

# result = final_chain.invoke({
#     "question": "스프링클러 설치시 참고해야하는 관련 법률 정보를 알려줘."
# })



# result = final_chain.invoke({
#     "question": "오늘 한국의 메인 뉴스 기사가 뭐야?"
# })

# result = final_chain.invoke({
#     "question": "영화 타이타닉 틀어줘."
# })

result = final_chain.invoke({
    "question": "법률 정보를 찾아볼 수 있는 사이트를 알려줘."
})

result.content




법률['법제처', '기본법령', '국가법령정보센터', '자치법규정보시스템', '한국법령정보원', '헌법재판소', '정부24', '입법 동향']

"['법제처', '기본법령', '국가법령정보센터', '자치법규정보시스템', '한국법령정보원', '헌법재판소', '정부24', '입법 동향']"